# Running computations in parallel


Consider a square and circle of radius one inside the square. 

Ratio between surface areas of circle and square

$\lambda = \frac{\pi}{4}$

Estimating λ: 
randomly sample points inside the square

Count how many fall inside the circle

Multiply this ratio by 4 for an estimate of $\pi$

In [1]:
import scala.util.Random
def mcCount(iter: Int): Int = {
val randomX = new Random
val randomY = new Random
var hits = 0
for (i <- 0 until iter) {
val x = randomX.nextDouble // in [0,1]
val y = randomY.nextDouble // in [0,1]
if (x*x + y*y < 1) hits= hits + 1
}
hits
}
def monteCarloPiSeq(iter: Int): Double = 4.0 * mcCount(iter) / iter

import scala.util.Random

defined function mcCount
defined function monteCarloPiSeq

In [4]:
monteCarloPiSeq(100000)

res3: Double = 3.1458

Here is the parallel version of computing the same estimate.

```scala
def monteCarloPiPar(iter: Int): Double = {
val ((pi1, pi2), (pi3, pi4)) = parallel(
parallel(mcCount(iter/4), mcCount(iter/4)),
parallel(mcCount(iter/4), mcCount(iter - 3*(iter/4))))
4.0 * (pi1 + pi2 + pi3 + pi4) / iter
}
```

All computations are happening in parallel and independent.

# First Class Tasks

Let's now describe a more flexible construct for describing parallel computation. Consider

```scala
val (v1, v2) = parallel(e1, e2)
```

we can write alternatively using the `task` construct:

```scala
val t1 = task(e1)
val t2 = task(e2)
val v1 = t1.join
val v2 = t2.join
t1
t2

```

`
t = task(e)` starts computation `e` "in the background"

* `t` is a task, which performs computation of `e`
* current computation proceeds in parallel with `t`
* to obtain the result of `e`, use `t.join`
* `t.join` blocks and waits until the result is computed
* subsequent `t.join` calls quickly return the `s`

Here is a minimal interface for tasks:

```scala
def task(c: => A) : Task[A]
trait Task[A] {
def join: A
}

```

`task` and `join` establish maps between computations and tasks
In terms of the value computed the equation `task(e).join==e` holds

We can omit writing `.join` if we also define an implicit conversion.

```scala
implicit def getJoin[T](x:Task[T]): T = x.join
```

We have seen four-way parallel p-norm:

```scala
val ((part1, part2),(part3,part4)) =
parallel(parallel(sumSegment(a, p, 0, mid1),
sumSegment(a, p, mid1, mid2)),
parallel(sumSegment(a, p, mid2, mid3),
sumSegment(a, p, mid3, a.length)))
power(part1 + part2 + part3 + part4, 1/p)
```

Here is essentially the same computation expressed using task:
```scala
val t1 = task {sumSegment(a, p, 0, mid1)}
val t2 = task {sumSegment(a, p, mid1, mid2)}
val t3 = task {sumSegment(a, p, mid2, mid3)}
val t4 = task {sumSegment(a, p, mid3, a.length)}
power(t1 + t2 + t3 + t4, 1/p)
```

Notice the implicit conversion of `t1`, `t2`, `t3` and `t4` in the last statement.

Suppose you are allowed to use task
Implement parallel construct as a method using task

```scala
def parallel[A, B](cA: => A, cB: => B): (A, B) = {
val tB: Task[B] = task { cB }
val tA: A = cA
(tA, tB.join)
}
```

Suppose we want to compute computation `A` and `B`. Using `task` construct start a computation in parallel, and we continue in parallel
in thread of this function.

-------------------
What is wrong with the following definitions?

```scala
def parallelWrong[A, B](cA: => A, cB: => B): (A, B) = {
val tB: B = (task { cB }).join
val tA: A = cA
(tA, tB.join)
}
```

Although, above defintion compiles it does not the benefit of parallel computation in as task `tB` completed (immediatly called join) before task `tA`.

# How fast are parallel programs?


**Performance**: a key motivation for parallelism

How to estimate it?

* empirical measurement
* asymptotic analysis
Asymptotic analysis is important to understand how algorithms scale when:
* inputs get larger
* we have more hardware parallelism available


We examine worst-case (as opposed to average) bounds

You have previously learned how to concisely characterize behavior of
sequential programs using the number of operations they perform as a
function of arguments.


* inserting into an integer into a sorted linear list takes time $\mathcal{O}(n)$, for list storing $n$ integers
* inserting into an integer into a balanced binary tree of $n$ integers takes time $\mathcal{O}(log n)$, for tree storing $n$ integers.


Let us review these techniques by applying them to our sum segment
example